In [ ]:
from tvsclib.strict_system import StrictSystem
from tvsclib.stage import Stage

from tvsclib.mixed_system import MixedSystem
import numpy as np
import scipy.linalg as linalg
import matplotlib.pyplot as plt
import scipy.linalg 
import scipy.stats 
import tvsclib.utils as utils
import tvsclib.math as math
from tvsclib.approximation import Approximation
from tvsclib.identification import identify


import setup_plots
import move

import torchvision.models as models
import torch


from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.toeplitz_operator import ToeplitzOperator
import time


In [ ]:
setup_plots.setup()
plt.rcParams['figure.dpi'] = 150

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

def plot_moves(sys_move,input_dims,output_dims,fs,text_ylabel=" "):
    w = setup_plots.textwidth
    #fig, ax = plt.subplots(1,1,figsize=(1*w, .5*w))
    #fig, (ax,axf) = plt.subplots(1,2,figsize=(1*w, .5*w),gridspec_kw={'width_ratios':[2,1]})
    fig = plt.figure(figsize=(1*w, .5*w)) 
    ax = fig.add_axes([0.1,0.2,0.65,0.9]) #[left, bottom, width, height]
    axf = fig.add_axes([0.85,0.35,0.3,0.45]) #[left, bottom, width, height]

    utils.show_system(sys_move,ax=ax)
    y_lim = ax.get_ylim()
    x_lim = ax.get_xlim()
    ax.xaxis.set_ticks_position('top')

    divider = make_axes_locatable(ax)
    ax_dimsin = divider.append_axes("top", 0.68, pad=0.1, sharex=ax)
    ax_dimsout = divider.append_axes("left", 0.68, pad=0.1, sharey=ax)

    # make some labels invisible
    ax_dimsin.xaxis.set_tick_params(labelbottom=False)
    ax_dimsout.yaxis.set_tick_params(labelright=False)

    N = input_dims.shape[1]

    ax_dimsin.invert_yaxis()

    angl = np.array([0.1,-0.1]*N)#add vector to make the lines slightly angled

    din_cum=np.cumsum(input_dims,axis=0)
    dout_cum=np.cumsum(output_dims,axis=0)
    for i in range(dout_cum.shape[0]-1):
        ax_dimsout.plot(np.repeat(np.arange(dout_cum.shape[1]+1),2)[1:-1]+angl,
                        np.repeat(dout_cum[i,:],2)-0.5,\
                       linestyle='solid',color='C0')

    din_cum=np.cumsum(input_dims,axis=0)
    for i in range(din_cum.shape[0]-1):
        ax_dimsin.plot(np.repeat(din_cum[i,:],2)-0.5,
                       np.repeat(np.arange(din_cum.shape[1]+1),2)[1:-1]+angl,\
                      linestyle='solid',color='C0')


    ax_dimsout.xaxis.set_ticks_position('top')
    ax_dimsout.yaxis.set_ticks_position('right')
    ax_dimsout.yaxis.set_tick_params(labelright=False)

    spacing =2#how manx iteration maRKERS
    ax_dimsin.set_yticks(np.arange(1,N,spacing))
    ax_dimsout.set_xticks(np.arange(1,N,spacing))



    #ax_dimsin.set_xticks(np.arange(3,48,3)-0.5)
    #ax_dimsout.set_yticks(np.arange(3,48,3)-0.5)

    ax_dimsin.grid()
    ax_dimsout.grid()
    ax_dimsout.set_xlim((0,N))
    ax_dimsin.set_ylim((N,0))   
    ax.set_ylim(y_lim)
    ax.set_xlim(x_lim)
    
    ax.set_xticks(np.hstack([0,np.cumsum(sys_move.dims_in)])-0.5)
    ax.set_yticks(np.hstack([0,np.cumsum(sys_move.dims_out)])-0.5)
    ax.set_xticklabels([' '])
    ax.set_yticklabels([' '])

    offset = 0.1


    ax.text(0,0,r'Iteration$\quad\quad\quad$.',rotation=-45,\
                             horizontalalignment='right', verticalalignment='center',rotation_mode='anchor')

    
    axf.plot(fs)
    axf.grid()
    axf.set_xlabel('Iteration')
    axf.set_ylabel(text_ylabel)
    axf.ticklabel_format(axis='y',scilimits=(0,0))
    axf.set_xticks(np.arange(1,N,spacing))
    axf.set_xlim((0,N-1))
    

In [ ]:
def get_mobilenet_target_mats():
    target_mats = []
    # Load the model
    model = models.mobilenet_v2(pretrained=True)
    # Put moel into eval mode
    model.eval()
    for layer in model.classifier:
        if isinstance(layer, torch.nn.Linear):
            # Obtain the weights of this layer
            weights = layer.weight.detach().numpy()
            target_mats.append(weights)
    return target_mats
mat_mobilenet = get_mobilenet_target_mats()[0]

In [ ]:
mat_mobilenet.shape

In [ ]:
stages = 10

#set the dims
d_in = mat_mobilenet.shape[1]
boundaries = d_in/stages*np.arange(stages+1)
boundaries = np.round(boundaries).astype(int)
dims_in = boundaries[1:]-boundaries[:-1]

d_out = mat_mobilenet.shape[0]
boundaries = d_out/stages*np.arange(stages+1)
boundaries = np.round(boundaries).astype(int)
dims_out = boundaries[1:]-boundaries[:-1]

assert sum(dims_in)==d_in and sum(dims_out)==d_out

system = identify(mat_mobilenet, dims_in,dims_out,epsilon=2e-8)

#approx =Approximation(system)
print(system)

In [ ]:
def cost_computation(sigmas_causal,sigmas_anticausal,dims_in,dims_out):
    k = len(dims_in)
    dims_state_causal = np.zeros(k+1)
    dims_state_anticausal = np.zeros(k+1)
    #get the number of stages for each step
    for i in range(k-1):
        dims_state_causal[i+1] = np.count_nonzero(sigmas_causal[i]>eps)
        dims_state_anticausal[i+1] = np.count_nonzero(sigmas_anticausal[i]>eps)
    
    return math.cost(dims_in,dims_out,dims_state_causal,causal=True)\
            +math.cost(dims_in,dims_out,dims_state_anticausal,causal=False,include_D=False)

In [ ]:
#eps_max = np.linalg.svd(mat_mobilenet[int(np.floor(mat_mobilenet.shape[0]/2)):,:int(np.floor(mat_mobilenet.shape[1]/2))],compute_uv=False)[0]
eps_max = math.hankelnorm(mat_mobilenet,system.dims_in,system.dims_out)
eps = eps_max*0.25
print("eps:",eps)

m_in=np.ceil(30/1.5**np.arange(10)).astype(int)
m_out=m_in
sys_move,input_dims,output_dims,fs,sigmas = move.move(system,None,cost_computation,m_in=m_in,m_out=m_out,cost_global=True,
                                               return_sigmas=True,print_progress=True)
print("l=")
display(m_out)

In [ ]:
plot_moves(sys_move,input_dims,output_dims,fs,text_ylabel=r'$\text{f}_{\text{FLOP}}(\Sigma)$')

plt.savefig("move_example_mobilenet_comp.pdf",bbox="tight",bbox_inches = 'tight')
bbox = plt.gcf().get_tightbbox( plt.gcf().canvas.get_renderer()) 
print(bbox.width/setup_plots.textwidth)

## Compute the segmentations for different $K$s

In [ ]:
def compute_moved_systems(stages):
    #set the dims
    d_in = mat_mobilenet.shape[1]
    boundaries = d_in/stages*np.arange(stages+1)
    boundaries = np.round(boundaries).astype(int)
    dims_in = boundaries[1:]-boundaries[:-1]

    d_out = mat_mobilenet.shape[0]
    boundaries = d_out/stages*np.arange(stages+1)
    boundaries = np.round(boundaries).astype(int)
    dims_out = boundaries[1:]-boundaries[:-1]

    assert sum(dims_in)==d_in and sum(dims_out)==d_out

    system = identify(mat_mobilenet, dims_in,dims_out,epsilon=2e-8)
    sys_move,input_dims,output_dims,fs,sigmas = move.move(system,None,cost_computation,m_in=m_in,m_out=m_out,cost_global=True,
                                               return_sigmas=True,print_progress=False)
    
    approx = Approximation(sys_move,sigmas)
    system = approx.get_approxiamtion(eps)
    return (approx,system)

approxs = []
systems = []
Ks = [5,10,17]
for K in Ks:
    print("Stages:",K)
    ap,sy = compute_moved_systems(K)
    approxs.append(ap)
    systems.append(sy)

    

In [ ]:
w = setup_plots.textwidth
#plt.figure(figsize = [w,w*3/4])
fig, axes = plt.subplots(ncols=3 , figsize = [w,w/2.5],gridspec_kw={"left":0.02,"right":0.98,"bottom":0.1,"top":0.9})

for i in range(len(Ks)):
    ax=axes[i]
    utils.show_system(systems[i],ax=ax)
    

    ax.set_xticks(np.cumsum(systems[i].dims_in[:-1])-0.5)
    ax.set_yticks(np.cumsum(systems[i].dims_out)-0.5)
    ax.set_xticklabels([' '])
    ax.set_yticklabels([' '])
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    
    print("K=",Ks[i])
    print("  cost=",systems[i].cost())

plt.savefig("Mobilenet_diff_K.pdf",bbox="tight")


# Compute the cost vs error

In [ ]:
approx =Approximation(system)
approx_move =Approximation(sys_move)

N = 9 #number of points
alpha = np.linspace(0,1,N)

err_move =np.zeros_like(alpha)

eps = eps_max*alpha

def calc_values(approx,eps):
    costs =np.zeros_like(eps)
    err =np.zeros_like(eps)
    for i in range(len(eps)):
        approx_system=approx.get_approxiamtion(eps[i])
        matrix_approx = approx_system.to_matrix()
        err[i] = np.linalg.norm(matrix_approx-mat_mobilenet,ord=2)
        costs[i] = approx_system.cost()
    return err,costs

err_orig,cost_orig = calc_values(approx,eps)
err_move,cost_move = calc_values(approx_move,eps)

In [ ]:
w = 0.75*setup_plots.textwidth
fig, ax = plt.subplots(figsize=(w, 2/3*w))
plt.plot(cost_orig,err_orig,'1-',label='Initial segmentation')
plt.plot(cost_move,err_move,'2--',label='Adapted segmentation')
ylims = ax.get_ylim()
plt.vlines(mat_mobilenet.size,ylims[0],ylims[1],colors='0.4')
ax.set_ylim(ylims)
plt.grid()
plt.legend()

i = np.argmin(np.abs(alpha-0.25))
#plt.scatter([cost_orig[i],cost_move[i]],[err_orig[i],err_move[i]])


plt.xlabel("Number of multiplications")
plt.ylabel(r'$\| M-\tilde{T} \| $')

zoom_h = 1
zoom_w = 3e5
s = 5.2
axins = ax.inset_axes([0.45, 0.2,s*zoom_w/4e6,s*zoom_h/10])

axins.grid()
axins.plot(cost_orig,err_orig,'1-')
axins.plot(cost_move,err_move,'2--')

axins.set_xlim(cost_orig[i]-0.5*zoom_w, cost_orig[i]+0.5*zoom_w)
axins.set_ylim(err_orig[i]-0.5*zoom_h, err_orig[i]+0.5*zoom_h)
axins.set_xticklabels([])
axins.set_yticklabels([])
text = ax.text(cost_orig[0]-3e5,err_orig[0]+0.7, r'$\epsilon = 0$',
                  bbox={'facecolor': 'white',"edgecolor":"black", 'alpha': 0.5, 'pad': 0,"linewidth":0})
text = ax.text(cost_orig[-1]+1.1e5,err_orig[-1]-.2, r'$\epsilon = \|M\|_H$',
                  bbox={'facecolor': 'white',"edgecolor":"black", 'alpha': 0.5, 'pad': 0,"linewidth":0})

ax.indicate_inset_zoom(axins, edgecolor="black")
text = axins.text(cost_orig[i]-0.15*zoom_w,err_orig[i]+0.3*zoom_h, r'$\epsilon = \frac{1}{4} \|M\|_H$',
                  bbox={'facecolor': 'white',"edgecolor":"black", 'alpha': 0.5, 'pad': 0,"linewidth":0})

plt.savefig("move_example_mobilenet_error.pdf",bbox="tight",bbox_inches = 'tight')
bbox = plt.gcf().get_tightbbox( plt.gcf().canvas.get_renderer()) 
print(bbox.width/setup_plots.textwidth)

In [ ]:
np.linalg.norm(mat_mobilenet,ord=2)

In [ ]:
alpha[i]

In [ ]:
print("Cost original=",cost_orig[i])
print("Cost move=",cost_move[i])
print("Cost move/Cost orig=",cost_move[i]/cost_orig[i])

In [ ]:
print("Original errror=",err_orig[i])
print("New errror     =",err_move[i])

In [ ]:
sys_apr_quater = approx.get_approxiamtion(eps[i])
sys_apr_quater_move = approx_move.get_approxiamtion(eps[i])
print(eps[i])

dims_state_ref = [sys_apr_quater.causal_system.dims_state,sys_apr_quater.anticausal_system.dims_state]
dims_state = [sys_apr_quater_move.causal_system.dims_state,sys_apr_quater_move.anticausal_system.dims_state]

x = np.arange(len(dims_state[0]))  # the label locations
width = 0.35  # the width of the bars

w = setup_plots.textwidth
fig, axes = plt.subplots(2,1,figsize=(w, w/2),sharex=True)
for v in [0,1]:
    ax =axes[v]
    if v ==0:
        label = ['Initial',None]
        ax.set_ylabel(r"$d$")
    else:
        label = [None,'Adapted']
        ax.set_ylabel(r"$d^*$")
    rects1 = ax.bar(x - width/2, dims_state_ref[v], width, label=label[0])
    rects2 = ax.bar(x + width/2, dims_state[v], width, label=label[1])
    ax.grid()
    ax.set_ylim(0,120)
    if v ==0:
        ax.bar(0,0,label=r"$\Sigma ="+str(np.sum(dims_state_ref))+"$",color = "w")
        ax.bar(0,0,label=" ",color = "w")
    else:
        ax.bar(0,0,label=r"$\Sigma ="+str(np.sum(dims_state))+"$",color = "w")#linewidth=0
    

ax.set_xlim(0.4,9.6)
ax.set_xlabel(r"$k$")

fig.legend(loc='center right')
plt.subplots_adjust(right=0.75)

plt.savefig("move_example_mobilenet_state_dims.pdf",bbox="tight",bbox_inches = 'tight')
bbox = plt.gcf().get_tightbbox( plt.gcf().canvas.get_renderer()) 
print(bbox.width/setup_plots.textwidth)

In [ ]:
dim_sum = np.sum(dims_state,axis=0)
dim_sum_ref = np.sum(dims_state_ref,axis=0)
print(dim_sum,np.sum(dim_sum))
print(dim_sum_ref,np.sum(dim_sum_ref))

# AlexNet

In [ ]:
def get_AlexNet_target_mats():
    target_mats = []
    # Load the model
    model = models.alexnet(pretrained=True)
    # Put moel into eval mode
    model.eval()
    for layer in model.classifier:
        if isinstance(layer, torch.nn.Linear):
            # Obtain the weights of this layer
            weights = layer.weight.detach().numpy()
            target_mats.append(weights)
    return target_mats
mat_AlexNet = get_AlexNet_target_mats()[0]

In [ ]:
mat_AlexNet.shape

## get $K$

In [ ]:
M = min(mat_AlexNet.shape)
P = max(mat_AlexNet.shape)

K = np.logspace(0,np.log10(M))

gammas = np.logspace(-2,0,6)
for gamma in gammas:
    cost =(1/15)*K*P**2*gamma**2 + (1/3)*M*P*gamma + (1/3)*P**2*gamma + (M*P - 1/3*P**2*gamma**2)/K + (-1/3*M*P*gamma - 1/3*P**2*gamma)/K**2 + (4/15)*P**2*gamma**2/K**3
    plt.loglog(K,cost,label = str(gamma))

plt.legend()
plt.plot([K[0],K[-1]],[mat_AlexNet.size,mat_AlexNet.size],'k-')
plt.xlabel("$K$")
plt.ylabel("Approximated Cost")
plt.grid()

In [ ]:
system,sigmas = utils.load_system('AlexNet/system_start.npz',load_sigmas=True)
print(system)

In [ ]:
approx =Approximation(system,sigmas)
eps_max = np.max([np.max(s)for s in approx.sigmas_causal]+[np.max(s)for s in approx.sigmas_anticausal])
eps = eps_max*0.25
print("eps:",eps)
sys_approx = approx.get_approxiamtion(epsilon=eps/2)
print(sys_approx)
print(mat_AlexNet.size)
print(sys_approx.cost())

In [ ]:
data =np.load('AlexNet/data_move.npz')
input_dims = data["input_dims"]
output_dims=data["output_dims"]
fs = data["fs"]
sys_move,sigmas_move = utils.load_system('AlexNet/system_moved.npz',load_sigmas=True)

In [ ]:
plot_moves(sys_move,input_dims,output_dims,fs,text_ylabel=r'$\text{f}_{\text{FLOP}}(\Sigma)$')

plt.savefig("move_example_alexnet_comp.pdf",bbox="tight",bbox_inches = 'tight')
bbox = plt.gcf().get_tightbbox( plt.gcf().canvas.get_renderer()) 
print(bbox.width/setup_plots.textwidth)

In [ ]:
approx =Approximation(system,sigmas)
approx_move=Approximation(sys_move,sigmas_move)
approx_new =Approximation(system_new,sigmas_new)

N = 9 #number of points
alpha = np.linspace(0,1,N)

err_move =np.zeros_like(alpha)

eps = eps_max*alpha

def calc_values(approx,eps):
    costs =np.zeros_like(eps)
    err =np.zeros_like(eps)
    for i in range(len(eps)):
        approx_system=approx.get_approxiamtion(eps[i])
        matrix_approx = approx_system.to_matrix()
        err[i] = np.linalg.norm(matrix_approx-mat_AlexNet,ord=2)
        costs[i] = approx_system.cost()
    return err,costs

err_orig,cost_orig = calc_values(approx,eps)
err_move,cost_move = calc_values(approx_move,eps)
err_new,cost_new = calc_values(approx_new,eps)

In [ ]:
data = np.load('AlexNet/data_error.npz')
err_orig=data["err_orig"]
cost_orig=data["cost_orig"]
err_move=data["err_move"]
cost_move=data["cost_move"]
err_new=data["err_new"]
cost_new=data["cost_new"]
N=9
alpha = np.linspace(0,1,N)


In [ ]:
w = 0.75*setup_plots.textwidth
fig, ax = plt.subplots(figsize=(w, 2/3*w))
plt.plot(cost_orig,err_orig,'1-',label='Initial segmentation')
plt.plot(cost_move,err_move,'2--',label='Adapted segmentation')
plt.plot(cost_new,err_new,'3:',label='Recomputed system')
ylims = ax.get_ylim()
plt.vlines(mat_AlexNet.size,ylims[0],ylims[1],colors='0.4')
ax.set_ylim(ylims)
plt.grid()
plt.legend()

i = np.argmin(np.abs(alpha-0.25))
#plt.scatter([cost_orig[i],cost_move[i]],[err_orig[i],err_move[i]])


plt.xlabel("Number of multiplications")
plt.ylabel(r'$\| M-\tilde{T} \| $')

zoom_h = 0.3
zoom_w = 8e6
s = 7
axins = ax.inset_axes([0.45, 0.2,s*zoom_w/1.3e8/1.5,s*zoom_h/5])

axins.grid()
axins.plot(cost_orig,err_orig,'1-')
axins.plot(cost_move,err_move,'2--')
axins.plot(cost_new,err_new,'3:')

axins.set_xlim(cost_orig[i]-0.5*zoom_w, cost_move[i]+0.5*zoom_w)
axins.set_ylim(err_orig[i]-0.5*zoom_h, err_move[i]+0.5*zoom_h)
axins.set_xticklabels([])
axins.set_yticklabels([])

text = ax.text(cost_orig[0]-11e6,err_orig[0]+0.43, r'$\epsilon = 0$',
                  bbox={'facecolor': 'white',"edgecolor":"black", 'alpha': 0.5, 'pad': 1,"linewidth":0})
text = ax.text(cost_orig[-1]+3.5e6,err_orig[-1]-.4, r'$\epsilon = \|M\|_H$',
                  bbox={'facecolor': 'white',"edgecolor":"black", 'alpha': 0.5, 'pad': 0,"linewidth":0})

ax.indicate_inset_zoom(axins, edgecolor="black")
axins.text(cost_move[i]-0.11*zoom_w,err_move[i]+0.2*zoom_h, r'$\epsilon = \frac{1}{4} \|M\|_H$',
          bbox={'facecolor': 'white',"edgecolor":"black", 'alpha': 0.5, 'pad': 1,"linewidth":0})
plt.savefig("move_example_alexnet_error.pdf",bbox="tight",bbox_inches = 'tight')
bbox = plt.gcf().get_tightbbox( plt.gcf().canvas.get_renderer()) 
print(bbox.width/setup_plots.textwidth)

In [ ]:
eps_max

In [ ]:
0.3/eps_max

In [ ]:
eps_max*0.1

In [ ]:
i

In [ ]:
print("Cost original=",cost_orig[i])
print("Cost new=",cost_new[i])

print("Cost move/Cost orig=",cost_move[i]/cost_orig[i],(1-cost_move[i]/cost_orig[i])*100,"%")

print("Cost new/Cost orig=",cost_new[i]/cost_orig[i],(1-cost_new[i]/cost_orig[i])*100,"%")